# ENTREGA 2 - PROYECTO KAGGLE 2025-2



In [1]:
!pip install kaggle

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import json

!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

replicating local resources


In [16]:
os.environ['KAGGLE_CONFIG_DIR']='/content/'

In [17]:
data = {"username":"cmosquera15","key":"7370d240087006cabc28ef8f9fb3f99c"}
with open('kaggle.json','w') as file:
  json.dump(data, file, indent=4)

In [18]:
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [37]:
!unzip -q udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
!ls

replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
init.py      submission_example.csv
kaggle.json  test.csv
local	     train.csv
__pycache__  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
sample_data


In [38]:
!wc -l *.csv

   296787 submission_example.csv
   296787 test.csv
   692501 train.csv
  1286075 total


In [39]:
df = pd.read_csv("train.csv")
print("Shape:", df.shape)

Shape: (692500, 21)


## Base de Datos

In [40]:
df.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


## 1. Sustitución de valores faltantes (NaN) con etiquetas

In [41]:
df = df.copy()
for col in ['F_EDUCACIONMADRE', 'F_EDUCACIONPADRE', 'E_VALORMATRICULAUNIVERSIDAD']:
    df[col] = df[col].fillna('no info')
    df[col] = df[col].replace(['No sabe', 'No Aplica'], 'no info')

df[['F_EDUCACIONMADRE', 'F_EDUCACIONPADRE', 'E_VALORMATRICULAUNIVERSIDAD']].head()

,F_EDUCACIONMADRE,F_EDUCACIONPADRE,E_VALORMATRICULAUNIVERSIDAD
0,Postgrado,Técnica o tecnológica incompleta,Entre 5.5 millones y menos de 7 millones
1,Técnica o tecnológica incompleta,Técnica o tecnológica completa,Entre 2.5 millones y menos de 4 millones
2,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,Entre 2.5 millones y menos de 4 millones
3,Secundaria (Bachillerato) completa,no info,Entre 4 millones y menos de 5.5 millones
4,Primaria completa,Primaria completa,Entre 2.5 millones y menos de 4 millones


## 2. Conversión de rangos de matrícula a valores numéricos

In [42]:
df = df.copy()
cmap = {
    'Menos de 500 mil': 0.25,
    'Entre 500 mil y menos de 1 millón': 0.75,
    'Entre 1 millón y menos de 2.5 millones': 1.75,
    'Entre 2.5 millones y menos de 4 millones': 3.25,
    'Entre 4 millones y menos de 5.5 millones': 4.75,
    'Entre 5.5 millones y menos de 7 millones': 6.25,
    'Más de 7 millones': 7.75,
    'No pagó matrícula': 0,
    'no info': -1
}

df['E_VALORMATRICULAUNIVERSIDAD'] = df['E_VALORMATRICULAUNIVERSIDAD'].map(cmap)
df['E_VALORMATRICULAUNIVERSIDAD'].head()

,E_VALORMATRICULAUNIVERSIDAD
0,6.25
1,3.25
2,3.25
3,4.75
4,3.25


## 3. Normalización de variables numéricas

In [25]:
from sklearn.preprocessing import StandardScaler

df = df.copy()
num_cols = ['E_VALORMATRICULAUNIVERSIDAD', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])
df[num_cols].describe()

,E_VALORMATRICULAUNIVERSIDAD,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
count,6.925000e+05,6.925000e+05,6.925000e+05,6.925000e+05,6.925000e+05
mean,-4.395617e-17,-1.178013e-16,-1.450533e-16,2.177513e-15,-7.410679e-16
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,-1.638242e+00,-2.199542e+00,-2.781294e+00,-4.452543e+00,-3.869422e+00
25%,-8.849210e-01,-5.373728e-01,-5.134332e-01,-1.373936e-01,-1.163206e-01
50%,-4.544515e-01,-2.344159e-01,1.177168e-01,2.363596e-01,3.252207e-01
75%,8.369567e-01,3.714978e-01,5.242203e-01,5.251688e-01,5.901454e-01
max,2.128365e+00,3.179990e+00,2.428368e+00,9.838658e-01,1.016969e+00


## 4. Codificación one-hot de variables categóricas

In [26]:
df = df.copy()
cat_cols = ['F_EDUCACIONMADRE', 'F_EDUCACIONPADRE', 'F_ESTRATOVIVIENDA', 'E_PRGM_DEPARTAMENTO']
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)
df.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_TIENEINTERNET,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,...,E_PRGM_DEPARTAMENTO_NORTE SANTANDER,E_PRGM_DEPARTAMENTO_PUTUMAYO,E_PRGM_DEPARTAMENTO_QUINDIO,E_PRGM_DEPARTAMENTO_RISARALDA,E_PRGM_DEPARTAMENTO_SAN ANDRES,E_PRGM_DEPARTAMENTO_SANTANDER,E_PRGM_DEPARTAMENTO_SUCRE,E_PRGM_DEPARTAMENTO_TOLIMA,E_PRGM_DEPARTAMENTO_VALLE,E_PRGM_DEPARTAMENTO_VAUPES
0,904256,20212,ENFERMERIA,1.482661,Menos de 10 horas,Si,Si,Si,N,No,...,False,False,False,False,False,False,False,False,False,False
1,645256,20212,DERECHO,0.191253,0,No,Si,No,N,No,...,False,False,False,False,False,False,False,False,False,False
2,308367,20203,MERCADEO Y PUBLICIDAD,0.191253,Más de 30 horas,Si,Si,No,N,No,...,False,False,False,False,False,False,False,False,False,False
3,470353,20195,ADMINISTRACION DE EMPRESAS,0.836957,0,Si,Si,No,N,No,...,False,False,False,False,False,True,False,False,False,False
4,989032,20212,PSICOLOGIA,0.191253,Entre 21 y 30 horas,Si,Si,Si,N,No,...,False,False,False,False,False,False,False,False,False,False


## 5. Eliminación de columnas irrelevantes o redundantes

In [43]:
df = df.copy()
cols_to_drop = ['ID', 'PERIODO_ACADEMICO', 'F_TIENEINTERNET.1']
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])
df.head()

,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,ENFERMERIA,BOGOTÁ,6.25,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,DERECHO,ATLANTICO,3.25,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,MERCADEO Y PUBLICIDAD,BOGOTÁ,3.25,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,ADMINISTRACION DE EMPRESAS,SANTANDER,4.75,0,Estrato 4,Si,no info,Si,No,N,No,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,PSICOLOGIA,ANTIOQUIA,3.25,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


## 6. Conversión de variable objetivo a formato numérico

In [44]:
df = df.copy()
rend_map = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].map(rend_map)
df['RENDIMIENTO_GLOBAL'].value_counts()

,count
RENDIMIENTO_GLOBAL,
3,175619
0,172987
1,172275
2,171619


## 7. Validación de valores faltantes y estructura final

In [45]:
print("Valores faltantes por columna:")
print(df.isna().sum())

print("\nDimensiones finales del dataset:", df.shape)

Valores faltantes por columna:
E_PRGM_ACADEMICO                   0
E_PRGM_DEPARTAMENTO                0
E_VALORMATRICULAUNIVERSIDAD        0
E_HORASSEMANATRABAJA           30857
F_ESTRATOVIVIENDA              32137
F_TIENEINTERNET                26629
F_EDUCACIONPADRE                   0
F_TIENELAVADORA                39773
F_TIENEAUTOMOVIL               43623
E_PRIVADO_LIBERTAD                 0
E_PAGOMATRICULAPROPIO           6498
F_TIENECOMPUTADOR              38103
F_EDUCACIONMADRE                   0
RENDIMIENTO_GLOBAL                 0
INDICADOR_1                        0
INDICADOR_2                        0
INDICADOR_3                        0
INDICADOR_4                        0
dtype: int64

Dimensiones finales del dataset: (692500, 18)
